## Resumo dos relatórios dos diversos Vendors (utilizando LLM)

In [1]:
import openai
openai.__version__

import os
from openai import AzureOpenAI
from configparser import ConfigParser, ExtendedInterpolation
import httpx
import numpy as np
import pandas as pd
import PyPDF2
from datetime import datetime

config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('config-v1.x.ini', 'UTF-8')

http_client = httpx.Client(verify='petrobras-ca-root.pem')

client = AzureOpenAI(
    api_key=config['TST']['API_KEY_MODELOS_TEXTO'],
    api_version=config['OPENAI']['OPENAI_API_VERSION'],
    azure_endpoint=config['TST']['LITELLM_BASE_URL'],
    http_client=http_client
)

In [2]:
data_dados_basicos = {
'PKG_DESCRIPTION': ['API 610 CENTRIFUGAL PUMPS', 'DECK TROLLEY', 'FLARE SYSTEM', 'FRESH WATER CHLORINATION UNIT', 'NITROGEN GENERATOR UNITS', 'NON-API 610 CENTRIFUGAL PUMPS', 'PIG LAUNCHERS AND RECEIVERS', 'PROGRESSIVE CAVITY PUMPS', 'RECIPROCATING PUMPS', 'SEA WATER ELECTROCHLORINATION UNIT', 'SEA WATER LIFT PUMP', 'SHELL AND TUBE HEAT EXCHANGERS (Himile)', 'SHELL AND TUBE HEAT EXCHANGERS (Asvotec)', 'WATER INJECTION PUMPS', 'FRESH WATER MAKER FOR OIL DILUTION', 'PRINTED CIRCUIT HEAT EXCHANGER', 'OFFSHORE CRANE'],
'Discipline': ['ROE', 'ROE', 'ROE', 'STATIC', 'ROE', 'ROE', 'STATIC', 'ROE', 'ROE', 'STATIC', 'ROE', 'STATIC', 'STATIC', 'ROE', 'STATIC', 'STATIC', 'ROE'],
'Critical': ['', '', 'Critical', '', '', '', '', '', 'Critical', '', 'Super Critical', '', '', 'Super Critical', '', 'Critical', 'Critical'],
'System': ['1200,1223,5124,5125,5331,5115', '5266', '5412', '5122', '5241', '120N,1200,5115,5124,5125', '1244,1231,1210', '5412', '5115,5133', '5121', '5111', '1223', '1200,1223,1251,1350,5125,5135,5147,1225,1231', '1251', '5122', '1231,1252,1254', '5266'],
'Trigram': ['KD9', 'x', 'x', 'D5A', 'GK1', 'KD9', 'AP5', 'NZA', 'KFQ', 'D5A', 'FM2', 'HM9', 'ATI', 'SJA', 'HE1', 'HLT', 'ND1'],
'Vendor': ['KSB BRASIL LTDA', '', '', 'DE NORA DO BRASIL LTDA', 'GARDNER DENVER KOREA', 'KSB BRASIL LTDA', 'APPLIED ENGINEERING PTE LTD', 'Netzsch Asia Pacific Pte Ltd', 'PERONI POMPE SPA', 'DE NORA DO BRASIL LTDA', 'FRAMO AS', 'HIMILE MECHANICAL MANUFACTURING (SHANDONG) CO., LTD', 'ASVOTEC TERMOINDUSTRIAL LTDA', 'SULZER BRASIL SA', 'HITACHI AQUA TECH (HAQT)E1', 'HEATRIC DIVISION OF MEGGITT (UK) LIMITED', 'NATIONAL OILWELL VARCO NORWAY AS'],
'LD_Number': ['I-LD-3010.2S-1200-940-KD9-001_0','','','I-LD-3010.2S-5122-940-D5A-001_A','I-LD-3010.2S-5241-940-GK1-001_A','I-LD-3010.2S-120N-940-KD9-001_0','I-LD-3010.2S-1210-911-AP5-001_0','I-LD-3010.2S-5412-911-NZA-001_0','I-LD-3010.2S-1200-911-KFQ-001_0','I-LD-3010.2S-5121-940-D5A-001_0','I-LD-3010.2S-5111-940-FM2-001_A','I-LD-3010.2S-1223-940-HM9-001_A','I-LD-3010.2S-1200-940-ATI-001_0','I-LD-3010.2S-1251-911-SJA-001_B','I-LD-3010.2S-5122-940-HE1-101_0','','I-LD-3010.2S-5266-940-ND1-001_0'],
'MR_Number': ['I-RM-3010.2S-1200-311-S2N-002', 'I-RM-3010.2S-5266-620-S2N-001', 'I-RM-3010.2S-5412-583-S2N-001', 'I-RM-3010.2S-5122-560-S2N-042', 'I-RM-3010.2S-5241-470-S2N-001', 'I-RM-3010.2S-1200-311-S2N-001', 'I-RM-3010.2S-1210-296-S2N-001', 'I-RM-3010.2S-1200-313-S2N-001', 'I-RM-3010.2S-5133-312-S2N-001', 'I-RM-3010.2S-5121-560-S2N-040', 'I-RM-3010.2S-5111-311-S2N-001', 'I-RM-3010.2S-1200-451-S2N-011', 'I-RM-3010.2S-1200-451-S2N-011', 'I-RM-3010.2S-1251-311-S2N-001', 'I-RM-3010.2S-5122-580-S2N-040', 'I-RM-3010.2S-1200-459-S2N-032', 'I-RM-3010.2S-5266-631-S2N-001'],
'TBE_Number': ['I-PT-3010.2S-1200-311-S2N-002', 'I-PT-3010.2S-5266-620-S2N-001', 'I-PT-3010.2S-5412-583-S2N-001', 'I-PT-3010.2S-5122-560-S2N-042', 'I-PT-3010.2S-5241-470-S2N-001, I-PT-3010.2S-5241-470-S2N-002', 'I-PT-3010.2S-1200-311-S2N-001', 'I-PT-3010.2S-1210-296-S2N-018', 'I-PT-3010.2S-1200-313-S2N-001', 'I-PT-3010.2S-5133-312-S2N-001', 'I-PT-3010.2S-5121-560-S2N-040', 'I-PT-3010.2S-5111-311-S2N-001', 'I-PT-3010.2S-1200-451-S2N-011', 'I-PT-3010.2S-1200-451-S2N-011', 'I-PT-3010.2S-1251-311-S2N-001', 'I-PT-3010.2S-5122-580-S2N-040', 'I-PT-3010.2S-1200-459-S2N-032', 'I-PT-3010.2S-5266-631-S2N-001'],
'KOM_DATE': ['', '', '', '10/03/2025', '18/02/2025', '', '24/03/2025', '', '', '07/03/2025', '17-18/02/2025', '21/03/2025', '19/03/2025', '21-22/10/2025', '25/03/2025', '','24/03/2025'],
'PIM_DATE': ['', '', '', '', '01-02/04/2025', '', 's 08/2025', '', '', '', '', '', '', '17/01/2025', '', '', ''],
'DR_MOTORS_DATE': ['', '', '', '', '', '', '', '', '', '', '', '', '', '17/03/2025', '', '',''],
'PLAN_DELIVERY_DATE': ['', '', '', '28/02/2026', '30/10/2025 & 28/02/2026', '', '', '', '', '28/02/2026', '01/01/2027', '15/03/2026', '15/03/2026', '03/04/2026', '29/05/2026', '','10/12/2025 e 10/04/2027'],
}

df_db = pd.DataFrame(data_dados_basicos)

In [3]:
# Consulta API para listar modelos
models_resp = http_client.get(config['TST']['LITELLM_BASE_URL'] + '/models', headers={
    'api-key': config['TST']['API_KEY_MODELOS_TEXTO'],
    'Authorization': 'Bearer ' + config['TST']['API_KEY_MODELOS_TEXTO']
})
models_resp.raise_for_status()

<Response [200 OK]>

In [4]:
all_models = [m['id'] for m in models_resp.json()['data']]
len(all_models)

43

In [5]:
# Modelos de chat
models_chat = sorted(m for m in all_models if 'embedding' not in m)
models_chat

['claude-3-5-haiku-20241022-v1',
 'claude-3-7-sonnet-20250219',
 'claude-instant-v1',
 'claude-v2',
 'claude-v2.1',
 'claude-v3-haiku',
 'claude-v3-sonnet',
 'claude-v35-sonnet',
 'claude-v35-sonnet-v2',
 'command-light-v14',
 'command-r',
 'command-r-plus',
 'command-v14',
 'dall-e-3',
 'gpt-35-turbo-16k',
 'gpt-4o',
 'gpt-4o-2024-08-06',
 'gpt-4o-mini',
 'llama3-1-405b-instruct-v1',
 'llama3-1-70b-instruct-v1',
 'llama3-1-8b-instruct-v1',
 'llama3-2-11b-instruct-v1',
 'llama3-2-1b-instruct-v1',
 'llama3-2-3b-instruct-v1',
 'llama3-2-90b-instruct-v1',
 'llama3-3-70b-instruct-v1',
 'llama3-70b-instruct',
 'llama3-8b-instruct',
 'mistral-7b-instruct',
 'mistral-large',
 'mistral-small',
 'mixtral-8x7b-instruct',
 'nova-lite-v1',
 'nova-micro-v1',
 'nova-pro-v1',
 'stable-diffusion-xl-v1']

In [6]:
# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(messages, engine, max_response_tokens=500):
    response = client.chat.completions.create(
        model=engine,
        messages=messages,
        # temperature=0.5,
        max_tokens=max_response_tokens,
        # top_p=0.9,
    )
    return response.choices[0].message.content


# Defining a function to print out the conversation in a readable format
def print_conversation(messages):
    for message in messages:
        print(f"[{message['role'].upper()}]")
        print(message['content'])
        print()

### Extrair texto do PDF

In [7]:
def extrair_texto_pdf(caminho_pdf):
    """Extrai texto de um arquivo PDF"""
    with open(caminho_pdf, 'rb') as arquivo:
        leitor_pdf = PyPDF2.PdfReader(arquivo)
        texto_total = ''
        
        # Iterando sobre cada página do PDF
        for pagina in leitor_pdf.pages:
            texto_pagina = pagina.extract_text()
            texto_total += texto_pagina + '\n'
        
        return texto_total.strip()
    


In [8]:
def analisar_documento(caminho_pdf):
    """Analisa o documento PDF e retorna o texto extraído"""    
    
    base_system_message = "Você é um assistente especializado em suprimentos para grande projetos de construção de plataformas de petróleo. Você é capaz de responder perguntas sobre suprimentos, sobre os projetos e sobre os fornecedores."
    base_system_message += " Você também é capaz de realizar resumos sobre o status de vários pacotes de equipamentos."
    base_system_message += " Peço para você realizar um resumo em 3 linhas dos principais fatos relevantes e 3 linhas dos pontos de atenção (se houver)."
    base_system_message += " Não há necessidade de preencher as 3 linhas caso não exista informação relevante nos relatórios."
    system_message = f"{base_system_message.strip()}"

    user_message = extrair_texto_pdf(caminho_pdf)

    # Create the list of messages. role can be either "user" or "assistant" 
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]

    max_response_tokens = 500

    '''for model in models_chat:

        # skip stable diffusion
        if 'stable' in model:
            continue'''
    model = 'claude-v35-sonnet-v2'
    print(model.upper())
    try:
        response = send_message(
            messages,
            engine=model,
            max_response_tokens=max_response_tokens
        )
    
        print(response)
    except Exception as e:
        print(e)
        
    print('-' * 30)    

In [9]:
def processar_todos_documentos(pasta_relatorios, df_db):
    """Processa todos os documentos PDF e gera resumos por pacote"""
    # Criar pasta para os resumos se não existir
    pasta_resumos = os.path.join(os.path.dirname(pasta_relatorios), "resumos")
    os.makedirs(pasta_resumos, exist_ok=True)
    
    # Arquivo de log para registro de processamento
    log_file = os.path.join(pasta_resumos, f"log_processamento_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")
    
    with open(log_file, "w", encoding="utf-8") as log:
        log.write(f"Início do processamento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        # Obter lista de pacotes do dataframe
        pacotes = df_db["PKG_DESCRIPTION"].unique()
        
        for pacote in pacotes:
            try:
                # Verificar se existe arquivo PDF correspondente
                nome_arquivo = f"{pacote}.pdf"
                caminho_pdf = os.path.join(pasta_relatorios, nome_arquivo)
                
                if not os.path.exists(caminho_pdf):
                    log.write(f"ALERTA: Arquivo não encontrado para o pacote: {pacote}\n")
                    print(f"ALERTA: Arquivo não encontrado: {caminho_pdf}")
                    continue
                
                log.write(f"Processando pacote: {pacote}\n")
                print("-" * 80)
                print("-" * 80)
                print(f"Processando pacote: {pacote}")
                
                # Analisar o documento
                resumo = analisar_documento(caminho_pdf)
                if resumo is None:
                    log.write(f"ERRO: Falha ao analisar o documento para o pacote: {pacote}\n")
                    continue
                
                print(f"Resumo realizado com sucesso. Tamanho: {len(resumo)} caracteres")
                log.write(f"Resumo realizado com sucesso. Tamanho: {len(resumo)} caracteres\n")
                
                # Salvar resumo em arquivo de texto
                nome_arquivo_resumo = f"{pacote.replace('/', '_').replace('\\', '_')}_resumo.txt"
                caminho_resumo = os.path.join(pasta_resumos, nome_arquivo_resumo)
                
                with open(caminho_resumo, "w", encoding="utf-8") as f:
                    f.write(f"RESUMO DO RELATÓRIO: {pacote}\n")
                    f.write("=" * 80 + "\n\n")
                    f.write(resumo)
                
                log.write(f"Resumo gerado com sucesso para o pacote: {pacote}\n")
                log.write(f"Arquivo salvo em: {caminho_resumo}\n\n")
                print(f"Resumo salvo em: {caminho_resumo}")
                
            except Exception as e:
                log.write(f"ERRO ao processar o pacote {pacote}: {str(e)}\n\n")
                print(f"ERRO ao processar o pacote {pacote}: {str(e)}")
        
        log.write(f"\nFim do processamento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [10]:
# Caminho para a pasta com os relatórios
pasta_relatorios = r"C:\Users\ELXY\Documents\Codigos\Python\P84_85\ai_reports\files"

# Processar todos os documentos
processar_todos_documentos(pasta_relatorios, df_db)

print("Processamento concluído! Verifique a pasta de resumos e o arquivo de log.")

ALERTA: Arquivo não encontrado: C:\Users\ELXY\Documents\Codigos\Python\P84_85\ai_reports\files\API 610 CENTRIFUGAL PUMPS.pdf
ALERTA: Arquivo não encontrado: C:\Users\ELXY\Documents\Codigos\Python\P84_85\ai_reports\files\DECK TROLLEY.pdf
ALERTA: Arquivo não encontrado: C:\Users\ELXY\Documents\Codigos\Python\P84_85\ai_reports\files\FLARE SYSTEM.pdf
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Processando pacote: FRESH WATER CHLORINATION UNIT
CLAUDE-V35-SONNET-V2
Principais fatos relevantes:
1. O projeto está em fase inicial com 16,68% de progresso, focado na emissão de documentação técnica e aquisição antecipada de pacotes críticos.
2. Os principais itens críticos são Retificador-Transformador, Bombas, Painéis Elétricos, Estrutura do Skid e Instrumentação de Processo.
3. A estratégia foi alterada para antecipar os processos de aquisição visando mitigar riscos de atraso nas